In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

from pydub import AudioSegment, effects
from pydub.generators import WhiteNoise
from pydub.playback import play
from pydub.utils import mediainfo
import librosa
from librosa import display   
import noisereduce as nr
import IPython.display as ipd
from IPython.display import Audio
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pytz
import cv2

import tensorflow as tf
import tensorboard
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from json_tricks import dump, load

from DataModel import DataModel
from SERModel import SERModel
from Evaluation import Evaluation

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("TensorFlow version:  ", tf.__version__)
print("TensorBoard version: ", tensorboard.__version__)

tz = pytz.timezone('Asia/Hong_Kong')

### Experiment 33

In [ ]:
labelsToInclude = ['Anger', 'Frustration', 'Happiness', 'Neutral',  'Sadness']
mergeHappinessExcitement = True
splitDuration = 8
ignoreDuration = 2
transformByStft=True
hop_length = 2048
win_length = 2048
n_mels = 128
onehot = False

# Data Augmentation Parameters
# multiply = 3
# pitchScaleSemitonesOffset=3.0
# timeStretchOffset=0.2
# randomGainOffset=0.2
# addNoiseMaxFactor=0.2

mixDataModel5LabelsSplit4Ignore2Stft = DataModel(labelsToInclude=labelsToInclude,
                                                mergeHappinessExcitement=mergeHappinessExcitement,
                                                splitDuration=splitDuration,
                                                ignoreDuration=ignoreDuration,
                                                transformByStft=transformByStft,
                                                hop_length=hop_length,
                                                win_length=win_length,
                                                n_mels=n_mels,
                                                onehot=onehot)
mixDataModel5LabelsSplit4Ignore2Stft.extractIEMOCAPData()
mixDataModel5LabelsSplit4Ignore2Stft.extractEmoDBData()
mixDataModel5LabelsSplit4Ignore2Stft.processData()

In [ ]:
def training33(dataModel, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss):
  ySize = len(dataModel.labels_name)
  learning_rate = 0.00005
  decay = 0.0005
  input_shape = dataModel.x_train[0].shape

  cnnModel = SERModel(modelName,
                      experimentName,
                      ySize=ySize,
                      optimizerChoice=optimizer,
                      learning_rate=learning_rate,
                      decay=decay,
                      lossChoice=loss,
                      input_shape=input_shape,
                      activation=activation)
  
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent, early_stopping_patience=early_stopping_patience)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

  return evaluation

experimentName = "(Experiment 33) CNN Model A STFT RegL2 (100 Epochs) (IEMOCAP EmoDB) (No Data Aug) (5 Emotions with Merge and Split 8 Ignore 2 STFT 2048 2048) (00005 lr 0005 decay Stop 3)"
modelName = "cnnModelBstftRegL2"
epochs = 100
early_stopping_patience = 3
activation = 'relu'
optimizer = 'adam'
loss = 'scce'

evaluation = training33(mixDataModel5LabelsSplit4Ignore2Stft, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss)

### Experiment 34

In [ ]:
labelsToInclude = ['Anger', 'Frustration', 'Happiness', 'Neutral',  'Sadness']
mergeHappinessExcitement = True
splitDuration = 8
ignoreDuration = 2
transformByStft=True
hop_length = 1024
win_length = 2048
n_mels = 128
onehot = False

# Data Augmentation Parameters
# multiply = 3
# pitchScaleSemitonesOffset=3.0
# timeStretchOffset=0.2
# randomGainOffset=0.2
# addNoiseMaxFactor=0.2

mixDataModel5LabelsSplit4Ignore2Stft = DataModel(labelsToInclude=labelsToInclude,
                                                mergeHappinessExcitement=mergeHappinessExcitement,
                                                splitDuration=splitDuration,
                                                ignoreDuration=ignoreDuration,
                                                transformByStft=transformByStft,
                                                hop_length=hop_length,
                                                win_length=win_length,
                                                n_mels=n_mels,
                                                onehot=onehot)
mixDataModel5LabelsSplit4Ignore2Stft.extractIEMOCAPData()
mixDataModel5LabelsSplit4Ignore2Stft.extractEmoDBData()
mixDataModel5LabelsSplit4Ignore2Stft.processData()

In [ ]:
def training34(dataModel, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss):
  ySize = len(dataModel.labels_name)
  learning_rate = 0.00005
  decay = 0.0005
  input_shape = dataModel.x_train[0].shape

  cnnModel = SERModel(modelName,
                      experimentName,
                      ySize=ySize,
                      optimizerChoice=optimizer,
                      learning_rate=learning_rate,
                      decay=decay,
                      lossChoice=loss,
                      input_shape=input_shape,
                      activation=activation)
  
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent, early_stopping_patience=early_stopping_patience)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

  return evaluation

experimentName = "(Experiment 34) CNN Model A STFT RegL2 (100 Epochs) (IEMOCAP EmoDB) (No Data Aug) (5 Emotions with Merge and Split 8 Ignore 2 STFT 1024 2048) (00005 lr 0005 decay Stop 3)"
modelName = "cnnModelBstftRegL2"
epochs = 100
early_stopping_patience = 3
activation = 'relu'
optimizer = 'adam'
loss = 'scce'

evaluation = training34(mixDataModel5LabelsSplit4Ignore2Stft, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss)

### Experiment 35

In [ ]:
labelsToInclude = ['Anger', 'Frustration', 'Happiness', 'Neutral',  'Sadness']
mergeHappinessExcitement = True
splitDuration = 8
ignoreDuration = 2
transformByStft=True
hop_length = 512
win_length = 2048
n_mels = 128
onehot = False

# Data Augmentation Parameters
# multiply = 3
# pitchScaleSemitonesOffset=3.0
# timeStretchOffset=0.2
# randomGainOffset=0.2
# addNoiseMaxFactor=0.2

mixDataModel5LabelsSplit4Ignore2Stft = DataModel(labelsToInclude=labelsToInclude,
                                                mergeHappinessExcitement=mergeHappinessExcitement,
                                                splitDuration=splitDuration,
                                                ignoreDuration=ignoreDuration,
                                                transformByStft=transformByStft,
                                                hop_length=hop_length,
                                                win_length=win_length,
                                                n_mels=n_mels,
                                                onehot=onehot)
mixDataModel5LabelsSplit4Ignore2Stft.extractIEMOCAPData()
mixDataModel5LabelsSplit4Ignore2Stft.extractEmoDBData()
mixDataModel5LabelsSplit4Ignore2Stft.processData()

In [ ]:
def training35(dataModel, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss):
  ySize = len(dataModel.labels_name)
  learning_rate = 0.00005
  decay = 0.0005
  input_shape = dataModel.x_train[0].shape

  cnnModel = SERModel(modelName,
                      experimentName,
                      ySize=ySize,
                      optimizerChoice=optimizer,
                      learning_rate=learning_rate,
                      decay=decay,
                      lossChoice=loss,
                      input_shape=input_shape,
                      activation=activation)
  
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent, early_stopping_patience=early_stopping_patience)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

  return evaluation

experimentName = "(Experiment 35) CNN Model A STFT RegL2 (100 Epochs) (IEMOCAP EmoDB) (No Data Aug) (5 Emotions with Merge and Split 8 Ignore 2 STFT 512 2048) (00005 lr 0005 decay Stop 3)"
modelName = "cnnModelBstftRegL2"
epochs = 100
early_stopping_patience = 3
activation = 'relu'
optimizer = 'adam'
loss = 'scce'

evaluation = training35(mixDataModel5LabelsSplit4Ignore2Stft, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss)

### Experiment 36

In [ ]:
labelsToInclude = ['Anger', 'Frustration', 'Happiness', 'Neutral',  'Sadness']
mergeHappinessExcitement = True
splitDuration = 8
ignoreDuration = 2
transformByStft=True
hop_length = 256
win_length = 1024
n_mels = 128
onehot = False

# Data Augmentation Parameters
# multiply = 3
# pitchScaleSemitonesOffset=3.0
# timeStretchOffset=0.2
# randomGainOffset=0.2
# addNoiseMaxFactor=0.2

mixDataModel5LabelsSplit4Ignore2Stft = DataModel(labelsToInclude=labelsToInclude,
                                                mergeHappinessExcitement=mergeHappinessExcitement,
                                                splitDuration=splitDuration,
                                                ignoreDuration=ignoreDuration,
                                                transformByStft=transformByStft,
                                                hop_length=hop_length,
                                                win_length=win_length,
                                                n_mels=n_mels,
                                                onehot=onehot)
mixDataModel5LabelsSplit4Ignore2Stft.extractIEMOCAPData()
mixDataModel5LabelsSplit4Ignore2Stft.extractEmoDBData()
mixDataModel5LabelsSplit4Ignore2Stft.processData()

In [ ]:
def training36(dataModel, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss):
  ySize = len(dataModel.labels_name)
  learning_rate = 0.00005
  decay = 0.0005
  input_shape = dataModel.x_train[0].shape

  cnnModel = SERModel(modelName,
                      experimentName,
                      ySize=ySize,
                      optimizerChoice=optimizer,
                      learning_rate=learning_rate,
                      decay=decay,
                      lossChoice=loss,
                      input_shape=input_shape,
                      activation=activation)
  
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent, early_stopping_patience=early_stopping_patience)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

  return evaluation

experimentName = "(Experiment 36) CNN Model A STFT RegL2 (100 Epochs) (IEMOCAP EmoDB) (No Data Aug) (5 Emotions with Merge and Split 8 Ignore 2 STFT 256 1024) (00005 lr 0005 decay Stop 3)"
modelName = "cnnModelBstftRegL2"
epochs = 100
early_stopping_patience = 3
activation = 'relu'
optimizer = 'adam'
loss = 'scce'

evaluation = training36(mixDataModel5LabelsSplit4Ignore2Stft, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss)

### Experiment 37

In [ ]:
labelsToInclude = ['Anger', 'Frustration', 'Happiness', 'Neutral',  'Sadness']
mergeHappinessExcitement = True
splitDuration = 8
ignoreDuration = 2
transformByStft=True
hop_length = 128
win_length = 512
n_mels = 128
onehot = False

# Data Augmentation Parameters
# multiply = 3
# pitchScaleSemitonesOffset=3.0
# timeStretchOffset=0.2
# randomGainOffset=0.2
# addNoiseMaxFactor=0.2

mixDataModel5LabelsSplit4Ignore2Stft = DataModel(labelsToInclude=labelsToInclude,
                                                mergeHappinessExcitement=mergeHappinessExcitement,
                                                splitDuration=splitDuration,
                                                ignoreDuration=ignoreDuration,
                                                transformByStft=transformByStft,
                                                hop_length=hop_length,
                                                win_length=win_length,
                                                n_mels=n_mels,
                                                onehot=onehot)
mixDataModel5LabelsSplit4Ignore2Stft.extractIEMOCAPData()
mixDataModel5LabelsSplit4Ignore2Stft.extractEmoDBData()
mixDataModel5LabelsSplit4Ignore2Stft.processData()

In [ ]:
def training37(dataModel, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss):
  ySize = len(dataModel.labels_name)
  learning_rate = 0.00005
  decay = 0.0005
  input_shape = dataModel.x_train[0].shape

  cnnModel = SERModel(modelName,
                      experimentName,
                      ySize=ySize,
                      optimizerChoice=optimizer,
                      learning_rate=learning_rate,
                      decay=decay,
                      lossChoice=loss,
                      input_shape=input_shape,
                      activation=activation)
  
  history = cnnModel.fit(dataModel.x_train, dataModel.y_train, epochs, dataModel.validation_percent, early_stopping_patience=early_stopping_patience)
  
  evaluation = Evaluation(dataModel, cnnModel.resultDir, cnnModel.logDir, model=cnnModel.model)
  evaluation.evaluateAllHistory(history)
  
  print('')
  print('File Name: ' + cnnModel.logDir.split('/')[-1])

  return evaluation

experimentName = "(Experiment 37) CNN Model A STFT RegL2 (100 Epochs) (IEMOCAP EmoDB) (No Data Aug) (5 Emotions with Merge and Split 8 Ignore 2 STFT 128 512) (00005 lr 0005 decay Stop 3)"
modelName = "cnnModelBstftRegL2"
epochs = 100
early_stopping_patience = 3
activation = 'relu'
optimizer = 'adam'
loss = 'scce'

evaluation = training37(mixDataModel5LabelsSplit4Ignore2Stft, experimentName, modelName, epochs, early_stopping_patience, activation, optimizer, loss)